# Import Dependencies

In [38]:
import pandas as pd
from config import api_keyUSC, PG_key
from census import Census
from sqlalchemy import create_engine
import numpy as np

In [39]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect,func

In [40]:
import psycopg2

In [41]:
#import warnings
#from sqlalchemy import exc

# for warnings not included in regex-based filter below, just log
#warnings.filterwarnings(
  #"always", category=exc.RemovedIn20Warning
#)

# ASFIN

In [42]:
#import .csv file into DF
#note: currency values are in Thousands of Dollars

state_asfin_df = pd.read_csv ('2015_asfin_state_totals_transposed.csv')

In [43]:
state_asfin_df.set_index('State', inplace=True)
state_asfin_df.drop(['United States'], inplace = True)
state_asfin_df = state_asfin_df[['General expenditure','Education', 'Public welfare', 'Hospitals', 'Health','Police protection', 'Correction' ]]

state_asfin_df = state_asfin_df.rename(columns={'General expenditure':'Total General Expenditure','Education': 'Education Expediture', 'Public welfare':'Public Welfare Expenditure', 'Hospitals': 'Hospitals Expenditure', 'Health': 'Health Expenditure', 'Police protection':'Police Protection Expenditure', 'Correction':'Correction Expenditure'})

In [44]:
state_asfin_df.head()

,Total General Expenditure,Education Expediture,Public Welfare Expenditure,Hospitals Expenditure,Health Expenditure,Police Protection Expenditure,Correction Expenditure
State,,,,,,,
Alabama,"28,710,747","12,235,527","8,204,660","2,370,549","414,116","155,814","555,852"
Alaska,"11,580,917","2,703,274","2,113,211","68,984","283,698","161,965","340,726"
Arizona,"28,577,420","10,452,117","10,346,172","64,560","625,316","259,768","1,097,103"
Arkansas,"19,882,973","8,162,327","6,572,760","871,039","243,532","107,112","351,802"
California,"269,774,501","90,276,519","109,031,702","10,633,226","8,177,522","1,790,672","9,667,904"


# CENSUS

In [45]:
c = Census(api_keyUSC, year=2015)
census_data = c.acs5.get(("NAME", "B01003_001E", "B02001_003E",
                          "B02001_002E","B02001_005E","B03001_003E","B02001_008E",
                          "B17001_002E"), {'for': 'state:*'})

# Convert to DataFrame
census_df = pd.DataFrame(census_data)

# Column Reordering

census_df.drop('state', axis=1, inplace = True)
census_df = census_df.rename(columns={"NAME": "State",
                                      "B01003_001E": "Population",
                                      "B02001_003E": "Population Black",
                                      "B02001_002E": "Population White",
                                      "B02001_005E": "Population Asian",
                                      "B03001_003E": "Population Hispanic",
                                      "B02001_008E": "Population Mixed Race",
                                      "B17001_002E": "Poverty Count",
                                      })

# Add in Poverty Rate (Poverty Count / Population)
census_df["Poverty Rate"] = round(100 * \
    census_df["Poverty Count"].astype(
        int) / census_df["Population"].astype(int),2)

In [46]:
census_df = census_df.sort_values('State')
census_df.set_index('State', inplace=True)
census_df

,Population,Population Black,Population White,Population Asian,Population Hispanic,Population Mixed Race,Poverty Count,Poverty Rate
State,,,,,,,,
Alabama,4830620.00,1276544.00,3325464.00,59599.00,193492.00,81646.00,887260.00,18.37
Alaska,733375.00,25022.00,484250.00,42921.00,47808.00,61755.00,72957.00,9.95
Arizona,6641928.00,281576.00,5210335.00,200090.00,2014711.00,211845.00,1180690.00,17.78
Arkansas,2958208.00,459748.00,2307849.00,40336.00,203226.00,62537.00,553644.00,18.72
California,38421464.00,2265387.00,23747013.00,5261978.00,14750686.00,1734897.00,6135142.00,15.97
Colorado,5278906.00,213787.00,4446095.00,153467.00,1112586.00,183480.00,653969.00,12.39
Connecticut,3593222.00,370501.00,2778362.00,150670.00,526508.00,101919.00,366351.00,10.20
Delaware,926454.00,200481.00,642623.00,33643.00,80706.00,25266.00,108315.00,11.69
District of Columbia,647484.00,316447.00,260325.00,23822.00,65803.00,17327.00,110365.00,17.05


In [47]:
#merge together
state_df = pd.merge(census_df,state_asfin_df, left_index=True, right_index=True)
state_df

,Population,Population Black,Population White,Population Asian,Population Hispanic,Population Mixed Race,Poverty Count,Poverty Rate,Total General Expenditure,Education Expediture,Public Welfare Expenditure,Hospitals Expenditure,Health Expenditure,Police Protection Expenditure,Correction Expenditure
State,,,,,,,,,,,,,,,
Alabama,4830620.00,1276544.00,3325464.00,59599.00,193492.00,81646.00,887260.00,18.37,"28,710,747","12,235,527","8,204,660","2,370,549","414,116","155,814","555,852"
Alaska,733375.00,25022.00,484250.00,42921.00,47808.00,61755.00,72957.00,9.95,"11,580,917","2,703,274","2,113,211","68,984","283,698","161,965","340,726"
Arizona,6641928.00,281576.00,5210335.00,200090.00,2014711.00,211845.00,1180690.00,17.78,"28,577,420","10,452,117","10,346,172","64,560","625,316","259,768","1,097,103"
Arkansas,2958208.00,459748.00,2307849.00,40336.00,203226.00,62537.00,553644.00,18.72,"19,882,973","8,162,327","6,572,760","871,039","243,532","107,112","351,802"
California,38421464.00,2265387.00,23747013.00,5261978.00,14750686.00,1734897.00,6135142.00,15.97,"269,774,501","90,276,519","109,031,702","10,633,226","8,177,522","1,790,672","9,667,904"
Colorado,5278906.00,213787.00,4446095.00,153467.00,1112586.00,183480.00,653969.00,12.39,"26,917,961","10,988,746","7,927,080","827,482","427,722","178,366","1,022,917"
Connecticut,3593222.00,370501.00,2778362.00,150670.00,526508.00,101919.00,366351.00,10.20,"22,311,191","7,728,457","4,256,901","1,416,163","949,526","256,502","700,694"
Delaware,926454.00,200481.00,642623.00,33643.00,80706.00,25266.00,108315.00,11.69,"8,045,313","3,027,509","2,288,305","90,693","406,718","140,054","304,986"
Florida,19645772.00,3171108.00,14934702.00,509085.00,4660733.00,472082.00,3180109.00,16.19,"76,164,790","25,023,832","25,776,510","1,054,547","3,663,682","551,849","2,475,144"


In [48]:
#convert index to match the other datasets
us_state_abbrev = {
   'Alabama': 'AL',
   'Alaska': 'AK',
   'Arizona': 'AZ',
   'Arkansas': 'AR',
   'California': 'CA',
   'Colorado': 'CO',
   'Connecticut': 'CT',
   'Delaware': 'DE',
   'Florida': 'FL',
   'Georgia': 'GA',
   'Hawaii': 'HI',
   'Idaho': 'ID',
   'Illinois': 'IL',
   'Indiana': 'IN',
   'Iowa': 'IA',
   'Kansas': 'KS',
   'Kentucky': 'KY',
   'Louisiana': 'LA',
   'Maine': 'ME',
   'Maryland': 'MD',
   'Massachusetts': 'MA',
   'Michigan': 'MI',
   'Minnesota': 'MN',
   'Mississippi': 'MS',
   'Missouri': 'MO',
   'Montana': 'MT',
   'Nebraska': 'NE',
   'Nevada': 'NV',
   'New Hampshire': 'NH',
   'New Jersey': 'NJ',
   'New Mexico': 'NM',
   'New York': 'NY',
   'North Carolina': 'NC',
   'North Dakota': 'ND',
   'Ohio': 'OH',
   'Oklahoma': 'OK',
   'Oregon': 'OR',
   'Pennsylvania': 'PA',
   'Rhode Island': 'RI',
   'South Carolina': 'SC',
   'South Dakota': 'SD',
   'Tennessee': 'TN',
   'Texas': 'TX',
   'Utah': 'UT',
   'Vermont': 'VT',
   'Virginia': 'VA',
   'Washington': 'WA',
   'West Virginia': 'WV',
   'Wisconsin': 'WI',
   'Wyoming': 'WY'}

state_df.index = state_df.index.map(us_state_abbrev)

In [49]:
#verify dtypes 
state_df.dtypes

Population                       float64
Population Black                 float64
Population White                 float64
Population Asian                 float64
Population Hispanic              float64
Population Mixed Race            float64
Poverty Count                    float64
Poverty Rate                     float64
Total General Expenditure         object
Education Expediture              object
Public Welfare Expenditure        object
Hospitals Expenditure             object
Health Expenditure                object
Police Protection Expenditure     object
Correction Expenditure            object
dtype: object

In [50]:
#format values & convert from string to numeric
state_df = state_df.replace(',','', regex=True)
c = state_df.select_dtypes(object).columns
state_df[c] = state_df[c].apply(pd.to_numeric,errors='coerce')

#format the expenditure totals to true value (original values in thousands of $)
for column in state_df.iloc[:,8:15]:
        new_column = []
        for index, row in state_df.iterrows():
            new_value = ((row[column])* 1000)
            new_column.append(new_value)
        state_df[column]= new_column   
        pd.set_option('display.float_format','{:.2f}'.format)
state_df.head()

,Population,Population Black,Population White,Population Asian,Population Hispanic,Population Mixed Race,Poverty Count,Poverty Rate,Total General Expenditure,Education Expediture,Public Welfare Expenditure,Hospitals Expenditure,Health Expenditure,Police Protection Expenditure,Correction Expenditure
State,,,,,,,,,,,,,,,
AL,4830620.00,1276544.00,3325464.00,59599.00,193492.00,81646.00,887260.00,18.37,28710747000.00,12235527000.00,8204660000.00,2370549000.00,414116000.00,155814000.00,555852000.00
AK,733375.00,25022.00,484250.00,42921.00,47808.00,61755.00,72957.00,9.95,11580917000.00,2703274000.00,2113211000.00,68984000.00,283698000.00,161965000.00,340726000.00
AZ,6641928.00,281576.00,5210335.00,200090.00,2014711.00,211845.00,1180690.00,17.78,28577420000.00,10452117000.00,10346172000.00,64560000.00,625316000.00,259768000.00,1097103000.00
AR,2958208.00,459748.00,2307849.00,40336.00,203226.00,62537.00,553644.00,18.72,19882973000.00,8162327000.00,6572760000.00,871039000.00,243532000.00,107112000.00,351802000.00
CA,38421464.00,2265387.00,23747013.00,5261978.00,14750686.00,1734897.00,6135142.00,15.97,269774501000.00,90276519000.00,109031702000.00,10633226000.00,8177522000.00,1790672000.00,9667904000.00


In [51]:
#build out new columns with the true per capita $ amount state are spending in each category
# * by 1000 bc original spending values were in thousands 
for column in state_df.iloc[:,8:15]:
        new_column_percap = []
        new_column_percent = []
        for index, row in state_df.iterrows():
                per_capita = (row[column])/(row['Population'])
                new_column_percap.append(per_capita)
                percent = (row[column])/(row['Total General Expenditure'])*100
                new_column_percent.append(percent)
        state_df[column +' per capita'] = new_column_percap
        state_df[column +' Percentage of Budget'] = new_column_percent

In [133]:
#build out new columns with the percentage of the total population each race category is 
for column in state_df.iloc[:,1:6]:
        new_column = []
        for index, row in state_df.iterrows():
                percent = (row[column])/(row['Population'])*100
                new_column.append(percent)
                pd.set_option('display.float_format','{:.2f}'.format)
        state_df[column +' Percentage'] = new_column

In [69]:
pd.options.display.max_columns = None
state_df

,Population,Population Black,Population White,Population Asian,Population Hispanic,Population Mixed Race,Poverty Count,Poverty Rate,Total General Expenditure,Education Expediture,Public Welfare Expenditure,Hospitals Expenditure,Health Expenditure,Police Protection Expenditure,Correction Expenditure,Total General Expenditure per capita,Total General Expenditure Percentage of Budget,Education Expediture per capita,Education Expediture Percentage of Budget,Public Welfare Expenditure per capita,Public Welfare Expenditure Percentage of Budget,Hospitals Expenditure per capita,Hospitals Expenditure Percentage of Budget,Health Expenditure per capita,Health Expenditure Percentage of Budget,Police Protection Expenditure per capita,Police Protection Expenditure Percentage of Budget,Correction Expenditure per capita,Correction Expenditure Percentage of Budget,Population Black Percentage,Population White Percentage,Population Asian Percentage,Population Hispanic Percentage,Population Mixed Race Percentage
State,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AL,4830620.00,1276544.00,3325464.00,59599.00,193492.00,81646.00,887260.00,18.37,28710747000.00,12235527000.00,8204660000.00,2370549000.00,414116000.00,155814000.00,555852000.00,5943.49,100.00,2532.91,42.62,1698.47,28.58,490.73,8.26,85.73,1.44,32.26,0.54,115.07,1.94,26.43,68.84,1.23,4.01,1.69
AK,733375.00,25022.00,484250.00,42921.00,47808.00,61755.00,72957.00,9.95,11580917000.00,2703274000.00,2113211000.00,68984000.00,283698000.00,161965000.00,340726000.00,15791.26,100.00,3686.07,23.34,2881.49,18.25,94.06,0.60,386.84,2.45,220.85,1.40,464.60,2.94,3.41,66.03,5.85,6.52,8.42
AZ,6641928.00,281576.00,5210335.00,200090.00,2014711.00,211845.00,1180690.00,17.78,28577420000.00,10452117000.00,10346172000.00,64560000.00,625316000.00,259768000.00,1097103000.00,4302.58,100.00,1573.66,36.57,1557.71,36.20,9.72,0.23,94.15,2.19,39.11,0.91,165.18,3.84,4.24,78.45,3.01,30.33,3.19
AR,2958208.00,459748.00,2307849.00,40336.00,203226.00,62537.00,553644.00,18.72,19882973000.00,8162327000.00,6572760000.00,871039000.00,243532000.00,107112000.00,351802000.00,6721.29,100.00,2759.21,41.05,2221.87,33.06,294.45,4.38,82.32,1.22,36.21,0.54,118.92,1.77,15.54,78.02,1.36,6.87,2.11
CA,38421464.00,2265387.00,23747013.00,5261978.00,14750686.00,1734897.00,6135142.00,15.97,269774501000.00,90276519000.00,109031702000.00,10633226000.00,8177522000.00,1790672000.00,9667904000.00,7021.45,100.00,2349.64,33.46,2837.78,40.42,276.75,3.94,212.84,3.03,46.61,0.66,251.63,3.58,5.90,61.81,13.70,38.39,4.52
CO,5278906.00,213787.00,4446095.00,153467.00,1112586.00,183480.00,653969.00,12.39,26917961000.00,10988746000.00,7927080000.00,827482000.00,427722000.00,178366000.00,1022917000.00,5099.16,100.00,2081.63,40.82,1501.65,29.45,156.75,3.07,81.02,1.59,33.79,0.66,193.77,3.80,4.05,84.22,2.91,21.08,3.48
CT,3593222.00,370501.00,2778362.00,150670.00,526508.00,101919.00,366351.00,10.20,22311191000.00,7728457000.00,4256901000.00,1416163000.00,949526000.00,256502000.00,700694000.00,6209.24,100.00,2150.84,34.64,1184.70,19.08,394.12,6.35,264.25,4.26,71.38,1.15,195.00,3.14,10.31,77.32,4.19,14.65,2.84
DE,926454.00,200481.00,642623.00,33643.00,80706.00,25266.00,108315.00,11.69,8045313000.00,3027509000.00,2288305000.00,90693000.00,406718000.00,140054000.00,304986000.00,8683.99,100.00,3267.85,37.63,2469.96,28.44,97.89,1.13,439.01,5.06,151.17,1.74,329.20,3.79,21.64,69.36,3.63,8.71,2.73
FL,19645772.00,3171108.00,14934702.00,509085.00,4660733.00,472082.00,3180109.00,16.19,76164790000.00,25023832000.00,25776510000.00,1054547000.00,3663682000.00,551849000.00,2475144000.00,3876.90,100.00,1273.75,32.85,1312.06,33.84,53.68,1.38,186.49,4.81,28.09,0.72,125.99,3.25,16.14,76.02,2.59,23.72,2.40


# POLICE DEATHS

In [56]:
csv_file = "police_deaths.csv"
police_deaths_df = pd.read_csv(csv_file)
police_deaths_df.head()

,person,dept,eow,cause,cause_short,date,year,canine,dept_name,state
0,Constable Darius Quimby,"Albany County Constable's Office, NY","EOW: Monday, January 3, 1791",Cause of Death: Gunfire,Gunfire,1791-01-03,1791,False,Albany County Constable's Office,NY
1,Sheriff Cornelius Hogeboom,"Columbia County Sheriff's Office, NY","EOW: Saturday, October 22, 1791",Cause of Death: Gunfire,Gunfire,1791-10-22,1791,False,Columbia County Sheriff's Office,NY
2,Deputy Sheriff Isaac Smith,"Westchester County Sheriff's Department, NY","EOW: Thursday, May 17, 1792",Cause of Death: Gunfire,Gunfire,1792-05-17,1792,False,Westchester County Sheriff's Department,NY
3,Marshal Robert Forsyth,United States Department of Justice - United S...,"EOW: Saturday, January 11, 1794",Cause of Death: Gunfire,Gunfire,1794-01-11,1794,False,United States Department of Justice - United S...,US
4,Sheriff Robert Maxwell,"Greenville County Sheriff's Office, SC","EOW: Sunday, November 12, 1797",Cause of Death: Gunfire,Gunfire,1797-11-12,1797,False,Greenville County Sheriff's Office,SC


In [57]:
#Create filtered dataframe with specific columns
deaths_columns = ["year", "cause_short", "state"]
police_deaths_transformed = police_deaths_df[deaths_columns].copy()

In [58]:
#Rename the column headers
police_deaths_transformed = police_deaths_transformed.rename(columns = {"year" : "id"})
police_deaths_transformed.head()

,id,cause_short,state
0,1791,Gunfire,NY
1,1791,Gunfire,NY
2,1792,Gunfire,NY
3,1794,Gunfire,US
4,1797,Gunfire,SC


In [59]:
#Set index
police_deaths_transformed.set_index("id", inplace=True)
police_deaths_transformed.head()

,cause_short,state
id,,
1791,Gunfire,NY
1791,Gunfire,NY
1792,Gunfire,NY
1794,Gunfire,US
1797,Gunfire,SC


In [60]:
#loc data on year 2015
year_filtered= police_deaths_transformed.loc['2015']
year_filtered.head()

,cause_short,state
id,,
2015,Struck by vehicle,CO
2015,Heart attack,MO
2015,Vehicle pursuit,MD
2015,Automobile accident,TX
2015,Automobile accident,TX


In [130]:
#Create filter to get gunfire deaths only
pd.options.display.max_columns = None
gunfire_filtered= year_filtered.loc[year_filtered['cause_short'] == 'Gunfire']
gunfire_filtered

,cause_short,state
id,,
2015,Gunfire,GA
2015,Gunfire,PA
2015,Gunfire,US
2015,Gunfire,TR
2015,Gunfire,CA
2015,Gunfire,WI
2015,Gunfire,NY
2015,Gunfire,ID
2015,Gunfire,MS


In [131]:
#Group data by state

state_grouped = gunfire_filtered.groupby("state").count()
#Rename the column headers
state_grouped_df = state_grouped.rename(columns = {"cause_short" : "police_shooting_deaths"})

# format state names to remove leading space
state_grouped_df.index = state_grouped_df.index.str.lstrip()

# drop territories/tribes/incorrect data 
state_grouped_df.drop(['PR', 'TR', 'US'], inplace = True)

# show
state_grouped_df.head()

,police_shooting_deaths
state,
AR,1
CA,4
CO,1
FL,1
GA,1


# Police Killings

In [71]:
#Clean up the look of the df step one( show all columns)
#===============================================================
pd.options.display.max_columns = None



In [72]:
police_killings_df = pd.read_csv("police_killings(Dwayne)/police_killings.csv", encoding = "iso-8859-1")
display(police_killings_df)

,name,age,gender,raceethnicity,month,day,year,street address,city,state,latitude,longitude,state_fp,county_fp,tract_ce,geo_id,county_id,namelsad,lawenforcementagency,cause,armed,pop,share_white,share_black,share_hispanic,p_income,h_income,county_income,comp_income,county_bucket,nat_bucket,pov,urate,college
0,A'donte Washington,16,Male,Black,February,23,2015,Clearview Ln,Millbrook,AL,32.53,-86.36,1,51,30902,1051030902,1051,Census Tract 309.02,Millbrook Police Department,Gunshot,No,3779,60.5,30.5,5.6,28375,51367.00,54766,0.94,3.00,3.00,14.1,0.10,0.17
1,Aaron Rutledge,27,Male,White,April,2,2015,300 block Iris Park Dr,Pineville,LA,31.32,-92.43,22,79,11700,22079011700,22079,Census Tract 117,Rapides Parish Sheriff's Office,Gunshot,No,2769,53.8,36.2,0.5,14678,27972.00,40930,0.68,2.00,1.00,28.8,0.07,0.11
2,Aaron Siler,26,Male,White,March,14,2015,22nd Ave and 56th St,Kenosha,WI,42.58,-87.84,55,59,1200,55059001200,55059,Census Tract 12,Kenosha Police Department,Gunshot,No,4079,73.8,7.7,16.8,25286,45365.00,54930,0.83,2.00,3.00,14.6,0.17,0.15
3,Aaron Valdez,25,Male,Hispanic/Latino,March,11,2015,3000 Seminole Ave,South Gate,CA,33.94,-118.22,6,37,535607,6037535607,6037,Census Tract 5356.07,South Gate Police Department,Gunshot,Firearm,4343,1.2,0.6,98.8,17194,48295.00,55909,0.86,3.00,3.00,11.7,0.12,0.05
4,Adam Jovicic,29,Male,White,March,19,2015,364 Hiwood Ave,Munroe Falls,OH,41.15,-81.43,39,153,530800,39153530800,39153,Census Tract 5308,Kent Police Department,Gunshot,No,6809,92.5,1.4,1.7,33954,68785.00,49669,1.38,5.00,4.00,1.9,0.06,0.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,William Chapman II,18,Male,Black,April,22,2015,1098 Frederick Blvd,Portsmouth,VA,36.83,-76.34,51,740,211500,51740211500,51740,Census Tract 2115,Portsmouth Police Department,Gunshot,No,1640,40.9,53.8,0,25262,27418.00,46166,0.59,1.00,1.00,35.2,0.15,0.12
463,William Dick III,28,Male,Native American,April,4,2015,Bureau of Indian Affairs Rd 66,Tonasket,WA,48.71,-119.44,53,47,970400,53047970400,53047,Census Tract 9704,US Forest Service,Taser,Firearm,4154,74.5,0.4,20.2,18470,35608.00,40368,0.88,1.00,2.00,27.3,0.13,0.17
464,William Poole,52,Male,White,March,16,2015,130 Wedowee Ln,Gaston,NC,35.21,-81.24,37,71,31704,37071031704,37071,Census Tract 317.04,Gaston County Police Department,Gunshot,Firearm,3850,83.2,10.1,0.3,21175,38200.00,42017,0.91,2.00,2.00,28.5,0.26,0.07
465,Yuvette Henderson,38,Female,Black,February,3,2015,3800 Hollis St,Oakland,CA,37.83,-122.28,6,1,401700,6001401700,6001,Census Tract 4017,Emeryville Police Department,Gunshot,Firearm,2544,21.7,24.9,37.1,26971,63052.00,72112,0.87,2.00,4.00,23.9,0.07,0.40


In [73]:
#Changing the name of the dataframe
df = pd.DataFrame(police_killings_df)
df.head()

,name,age,gender,raceethnicity,month,day,year,street address,city,state,latitude,longitude,state_fp,county_fp,tract_ce,geo_id,county_id,namelsad,lawenforcementagency,cause,armed,pop,share_white,share_black,share_hispanic,p_income,h_income,county_income,comp_income,county_bucket,nat_bucket,pov,urate,college
0,A'donte Washington,16,Male,Black,February,23,2015,Clearview Ln,Millbrook,AL,32.53,-86.36,1,51,30902,1051030902,1051,Census Tract 309.02,Millbrook Police Department,Gunshot,No,3779,60.5,30.5,5.6,28375,51367.00,54766,0.94,3.00,3.00,14.1,0.10,0.17
1,Aaron Rutledge,27,Male,White,April,2,2015,300 block Iris Park Dr,Pineville,LA,31.32,-92.43,22,79,11700,22079011700,22079,Census Tract 117,Rapides Parish Sheriff's Office,Gunshot,No,2769,53.8,36.2,0.5,14678,27972.00,40930,0.68,2.00,1.00,28.8,0.07,0.11
2,Aaron Siler,26,Male,White,March,14,2015,22nd Ave and 56th St,Kenosha,WI,42.58,-87.84,55,59,1200,55059001200,55059,Census Tract 12,Kenosha Police Department,Gunshot,No,4079,73.8,7.7,16.8,25286,45365.00,54930,0.83,2.00,3.00,14.6,0.17,0.15
3,Aaron Valdez,25,Male,Hispanic/Latino,March,11,2015,3000 Seminole Ave,South Gate,CA,33.94,-118.22,6,37,535607,6037535607,6037,Census Tract 5356.07,South Gate Police Department,Gunshot,Firearm,4343,1.2,0.6,98.8,17194,48295.00,55909,0.86,3.00,3.00,11.7,0.12,0.05
4,Adam Jovicic,29,Male,White,March,19,2015,364 Hiwood Ave,Munroe Falls,OH,41.15,-81.43,39,153,530800,39153530800,39153,Census Tract 5308,Kent Police Department,Gunshot,No,6809,92.5,1.4,1.7,33954,68785.00,49669,1.38,5.00,4.00,1.9,0.06,0.40


In [74]:
#Using shape to count how many columns and rows are in the dateframe
#====================================================================
df.shape

(467, 34)

In [75]:
#Remove the columns that we found unneccessary for our analysis
#===============================================================================================================
df.drop(df.columns[[2,10,11,12,13,14,15,16,17,18,21,22,23,24,25,26,27,28,29,30,31,32,33]], axis=1, inplace=True)

In [76]:
#Check out results
#============================================
display(df)

,name,age,raceethnicity,month,day,year,street address,city,state,cause,armed
0,A'donte Washington,16,Black,February,23,2015,Clearview Ln,Millbrook,AL,Gunshot,No
1,Aaron Rutledge,27,White,April,2,2015,300 block Iris Park Dr,Pineville,LA,Gunshot,No
2,Aaron Siler,26,White,March,14,2015,22nd Ave and 56th St,Kenosha,WI,Gunshot,No
3,Aaron Valdez,25,Hispanic/Latino,March,11,2015,3000 Seminole Ave,South Gate,CA,Gunshot,Firearm
4,Adam Jovicic,29,White,March,19,2015,364 Hiwood Ave,Munroe Falls,OH,Gunshot,No
...,...,...,...,...,...,...,...,...,...,...,...
462,William Chapman II,18,Black,April,22,2015,1098 Frederick Blvd,Portsmouth,VA,Gunshot,No
463,William Dick III,28,Native American,April,4,2015,Bureau of Indian Affairs Rd 66,Tonasket,WA,Taser,Firearm
464,William Poole,52,White,March,16,2015,130 Wedowee Ln,Gaston,NC,Gunshot,Firearm
465,Yuvette Henderson,38,Black,February,3,2015,3800 Hollis St,Oakland,CA,Gunshot,Firearm


In [77]:
#Rename columns
#=============================================================================
df.rename(columns = {
                    'name': 'Name',
                    'age': 'Age',
                    'raceethnicity': 'Ethnicity',
                    'month': 'Month',
                    'day': 'Day',
                    'year': 'Year', 
                    'street address':'Street Address',
                    'city': 'City',
                    'state': 'State',
                    'cause': "Cause of Death",
                    'armed': 'Armed'
}, inplace=True)

In [78]:
#Check results
display(df)

,Name,Age,Ethnicity,Month,Day,Year,Street Address,City,State,Cause of Death,Armed
0,A'donte Washington,16,Black,February,23,2015,Clearview Ln,Millbrook,AL,Gunshot,No
1,Aaron Rutledge,27,White,April,2,2015,300 block Iris Park Dr,Pineville,LA,Gunshot,No
2,Aaron Siler,26,White,March,14,2015,22nd Ave and 56th St,Kenosha,WI,Gunshot,No
3,Aaron Valdez,25,Hispanic/Latino,March,11,2015,3000 Seminole Ave,South Gate,CA,Gunshot,Firearm
4,Adam Jovicic,29,White,March,19,2015,364 Hiwood Ave,Munroe Falls,OH,Gunshot,No
...,...,...,...,...,...,...,...,...,...,...,...
462,William Chapman II,18,Black,April,22,2015,1098 Frederick Blvd,Portsmouth,VA,Gunshot,No
463,William Dick III,28,Native American,April,4,2015,Bureau of Indian Affairs Rd 66,Tonasket,WA,Taser,Firearm
464,William Poole,52,White,March,16,2015,130 Wedowee Ln,Gaston,NC,Gunshot,Firearm
465,Yuvette Henderson,38,Black,February,3,2015,3800 Hollis St,Oakland,CA,Gunshot,Firearm


In [115]:
rds_connection_string = "postgres:postgres@localhost:5432/police_shootings"
engine = create_engine(f'postgresql://{rds_connection_string}')


In [96]:
inspector =inspect(engine)

In [97]:
inspector.get_table_names()


['police_killings', 'state_info', 'police_deaths']

In [98]:
#Using the inspector to print the column names within the 'Police_killings' table and its types
columns = inspector.get_columns('police_killings')
for column in columns:
    print(column["name"], column["type"])

ID INTEGER
Name VARCHAR
Age VARCHAR
Ethnicity VARCHAR
Month VARCHAR
Day INTEGER
Year INTEGER
Street Address VARCHAR
City VARCHAR
State VARCHAR
Cause of Death VARCHAR
Armed VARCHAR


In [99]:
#Load DataFrames into database 
df.to_sql(name='police_killings', con=engine, if_exists='append', index=False)


In [116]:
state_df.to_sql(name="state_info", con=engine, if_exists='append', index=True)


IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "state_info_pkey"
DETAIL:  Key ("State")=(AL) already exists.

[SQL: INSERT INTO state_info ("State", "Population", "Population Black", "Population White", "Population Asian", "Population Hispanic", "Population Mixed Race", "Poverty Count", "Poverty Rate", "Total General Expenditure", "Education Expediture", "Public Welfare Expenditure", "Hospitals Expenditure", "Health Expenditure", "Police Protection Expenditure", "Correction Expenditure", "Total General Expenditure per capita", "Total General Expenditure Percentage of Budget", "Education Expediture per capita", "Education Expediture Percentage of Budget", "Public Welfare Expenditure per capita", "Public Welfare Expenditure Percentage of Budget", "Hospitals Expenditure per capita", "Hospitals Expenditure Percentage of Budget", "Health Expenditure per capita", "Health Expenditure Percentage of Budget", "Police Protection Expenditure per ca

In [132]:
state_grouped_df.to_sql(name='police_deaths', con=engine, if_exists='append', index=True)

In [100]:
#Confirmation on Scrubbed
pd.read_sql_query('select * from police_killings', con=engine).head()
pd.read_sql_query('select * from police_killings', con=engine).head()

,ID,Name,Age,Ethnicity,Month,Day,Year,Street Address,City,State,Cause of Death,Armed
0,1,A'donte Washington,16,Black,February,23,2015,Clearview Ln,Millbrook,AL,Gunshot,No
1,2,Aaron Rutledge,27,White,April,2,2015,300 block Iris Park Dr,Pineville,LA,Gunshot,No
2,3,Aaron Siler,26,White,March,14,2015,22nd Ave and 56th St,Kenosha,WI,Gunshot,No
3,4,Aaron Valdez,25,Hispanic/Latino,March,11,2015,3000 Seminole Ave,South Gate,CA,Gunshot,Firearm
4,5,Adam Jovicic,29,White,March,19,2015,364 Hiwood Ave,Munroe Falls,OH,Gunshot,No


# 

In [82]:
#Create database connection
connection_string = "username:password@localhost:5432/shootings"
engine = create_engine(f'postgresql://{connection_string}')

In [83]:
#confirm table
engine.table_names()

<ipython-input-83-208f519aad61>:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['police_deaths']

In [85]:
dataframe.to_sql(name='table name', con=engine, if_exists='append', index= True)